# Q2 — Temporal Trends (Monthly, Weekday/Weekend, Category Trends)
**Source:** `violations_routes_filtered.csv`
**Scope:** M101, M60+, M15+; `is_exempt == True`

In [ ]:
import pandas as pd, numpy as np
from pathlib import Path

DATA = "/mnt/data/violations_routes_filtered.csv"
usecols = ["Bus Route ID","Datetime","is_exempt","Violation Status"]
df = pd.read_csv(DATA, usecols=usecols, low_memory=True)
df.columns = [c.strip() for c in df.columns]

def route_tag(r):
    if pd.isna(r): return None
    r = str(r).upper().strip()
    if r.startswith("M101"): return "M101"
    if r.startswith("M60"):  return "M60+" if ("SBS" in r or "+" in r or "-SBS" in r or "-" in r) else "M60"
    if r.startswith("M15"):  return "M15+" if (("SBS" in r) or ("+" in r) or ("-SBS" in r)) else "M15"
    return r

df["route_tag"] = df["Bus Route ID"].apply(route_tag)
df = df[df["route_tag"].isin(["M101","M60+","M15+"])]
df["is_exempt"] = df["is_exempt"].astype(str).str.lower().isin(["true","1","t","yes","y"])

df["Datetime"] = pd.to_datetime(df["Datetime"], errors="coerce", infer_datetime_format=True)
df["month"] = df["Datetime"].dt.to_period("M").astype(str)

ex = df[df["is_exempt"]==True].copy()

monthly_total = df.groupby(["route_tag","month"]).size().rename("total_violations").reset_index()
monthly_ex = ex.groupby(["route_tag","month"]).size().rename("exempt_violations").reset_index()
monthly = monthly_total.merge(monthly_ex, on=["route_tag","month"], how="left").fillna({"exempt_violations":0})
monthly["exempt_share_pct"] = (monthly["exempt_violations"]/monthly["total_violations"]*100).round(2)
display(monthly.head(36))

ex["weekday"] = ex["Datetime"].dt.dayofweek
ex["is_weekday"] = ex["weekday"] < 5
wkd = (ex.groupby(["route_tag","is_weekday","Violation Status"]).size()
       .rename("count").reset_index())
wkd["pct"] = (wkd["count"]/wkd.groupby(["route_tag","is_weekday"])["count"].transform("sum")*100).round(2)
display(wkd.head(36))

cat_month = (ex.groupby(["route_tag","month","Violation Status"]).size().rename("count").reset_index())
cat_month["month_total"] = cat_month.groupby(["route_tag","month"])["count"].transform("sum")
cat_month["share_pct"] = (cat_month["count"]/cat_month["month_total"]*100).round(2)
display(cat_month.head(36))

def period_delta(df_in):
    out = []
    for r, g in df_in.groupby("route_tag"):
        g2 = g.sort_values("month")
        months = g2["month"].unique()
        if len(months) < 6:
            continue
        first3, last3 = months[:3], months[-3:]
        a_first = g2[g2["month"].isin(first3)].groupby("Violation Status")["share_pct"].mean()
        a_last  = g2[g2["month"].isin(last3)].groupby("Violation Status")["share_pct"].mean()
        for s in sorted(set(a_first.index).union(a_last.index)):
            out.append({
                "route_tag": r,
                "Violation Status": s,
                "avg_share_first3": round(float(a_first.get(s, 0.0)),2),
                "avg_share_last3":  round(float(a_last.get(s, 0.0)),2),
                "delta_points":     round(float(a_last.get(s, 0.0) - a_first.get(s, 0.0)),2)
            })
    return pd.DataFrame(out).sort_values(["route_tag","delta_points"], ascending=[True, False])

cat_deltas = period_delta(cat_month); display(cat_deltas)

outdir = Path("/mnt/data/q2_notebooks/outputs"); outdir.mkdir(parents=True, exist_ok=True)
monthly.to_csv(outdir/"monthly_series.csv", index=False)
wkd.to_csv(outdir/"weekday_weekend_status_mix.csv", index=False)
cat_month.to_csv(outdir/"category_share_trends.csv", index=False)
cat_deltas.to_csv(outdir/"category_share_trend_deltas.csv", index=False)
print("Saved outputs to", outdir)